In [61]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime

In [ ]:
Left = 1600
Right = 1899
contestID = 2023

In [ ]:
page = requests.get(f'https://codeforces.com/api/contest.standings?contestId={contestID}&from=1&count=50000&showUnofficial=false')
Start = page.json()['result']['contest']['startTimeSeconds']

In [64]:
changes = requests.get(f'https://codeforces.com/api/contest.ratingChanges?contestId={contestID}')
changes = pd.DataFrame(changes.json()['result'],columns=['contestId','contestName','handle','rank','ratingUpdateTimeSeconds','oldRating','newRating'])
changes.drop(columns=['contestId','contestName','ratingUpdateTimeSeconds'],inplace=True)
changes.set_index('handle',inplace=True)
changes.head()

,rank,oldRating,newRating
handle,,,
jiangly,1,3839,3904
maroonrk,2,3223,3350
potato167,3,2921,3091
jeroenodb,4,2820,2999
kotatsugame,5,2917,3055


In [65]:
### Preparing the map for each problem
mp = {}
indices = []
for problem in page.json()['result']['problems']:
    mp[problem['index']] = []
    indices.append(problem['index'])
mp

{'A': [], 'B': [], 'C': [], 'D': [], 'E': [], 'F': []}

In [81]:
for row in page.json()['result']['rows']:
    try :
        handle = row['party']['members'][0]['handle']
        cur_rate = changes.loc[handle]['oldRating']
        
        if cur_rate > Right or cur_rate < Left:
            continue

        probs = pd.DataFrame(data=row['problemResults'],columns=['points','rejectedAttemptCount','type','bestSubmissionTimeSeconds'])
        probs = pd.concat([probs,pd.Series(indices,name='index')],axis=1)
        probs.set_index('index',inplace=True)
        probs = probs[probs.points > 0]
        probs.drop(columns=['points','rejectedAttemptCount','type'],inplace=True)
        last = 0
        for ind in probs.sort_values(by='bestSubmissionTimeSeconds').index:
            cur = probs.loc[ind]['bestSubmissionTimeSeconds']
            mp[ind].append((cur-last)/60)
            last = cur
        ###print(handle,cur_rate)
    except:
        pass

In [83]:
for key in mp.keys():
    print(key)
    try:
        print(pd.Series(mp[key]).describe().astype(dtype='int'))
    except:
        print('No one solved this problem!')
    print()

A
count    52
mean      8
std       4
min       1
25%       4
50%       7
75%      10
max      17
dtype: int64

B
count    54
mean     10
std       3
min       1
25%       8
50%      10
75%      12
max      19
dtype: int64

C
count    54
mean     34
std       9
min      17
25%      26
50%      33
75%      41
max      56
dtype: int64

D
count    20
mean     46
std      11
min      24
25%      41
50%      44
75%      54
max      67
dtype: int64

E
count     4
mean     50
std      32
min      22
25%      22
50%      50
75%      79
max      79
dtype: int64

F
No one solved this problem!

